In [ ]:
import time
import os
import psutil
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        tps = COUNT/total_time
        print("The Tps: %s Item/s" % tps)
    return int_time

In [ ]:
df_train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
COUNT = len(df_train)

In [ ]:
length=0
ct=0
for item in df_train['overview'].apply(str).to_list():
    length+=len(item)
    ct+=1
print('Avg char length of text column:',length/ct)

In [ ]:
import h2o
from h2o.estimators import H2OWord2vecEstimator, H2OGradientBoostingEstimator
documents = df_train['overview'].to_list()
doc_ids = list(range(len(documents)))
@count_time
def calcu_h2o():
    h2o.init()
    input_frame = h2o.H2OFrame(OrderedDict([('DocID', doc_ids), ('Document', documents)]),
                                    column_types=['numeric', 'string'])
    STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can",
                  "lines","re","what","there","all","we","one","the",
                  "a","an","of","or","in","for","by","on","but","is",
                  "in","a","not","with","as","was","if","they","are",
                  "this","and","it","have","from","at","my","be","by",
                  "not","that","to","from","com","org","like","likes",
                  "so"]
    # Make the 'tokenize' function:
    def tokenize(sentences, stop_word = STOP_WORDS):
        tokenized = sentences.tokenize("\\W+")
        tokenized_lower = tokenized.tolower()
        tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
        tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
        tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
        return tokenized_words

    words = tokenize(input_frame["Document"])

    # Build word2vec model:
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
    w2v_model.train(training_frame=words)

    # Find synonyms for the words "teacher":
    w2v_model.find_synonyms("teacher", count = 5)

    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
# In [4]:
calcu_h2o()